In [1]:
import pandas as pd

In [ ]:
!pip install  nba_api

In [8]:
from nba_api.stats.static import teams

nba_teams = teams.get_teams()

# We select the LA Clippers and we extract the team id
clippers = [team for team in nba_teams if team['abbreviation'] == 'LAC'][0]
clippers_id = clippers['id'] 
print(f"clippers_id:{clippers_id}")

clippers_id:1610612746


In [9]:
# We extract all 2024-25 season Clippers' games
from nba_api.stats.endpoints import leaguegamefinder

gamefinder = leaguegamefinder.LeagueGameFinder(team_id_nullable=clippers_id,
                                            season_nullable='2024-25',
                                            season_type_nullable='Regular Season')

games = gamefinder.get_normalized_dict()['LeagueGameFinderResults']

In [10]:
# Function that returns the opponent abbreviation team from the matchup. Ex: LAC vs. OKC => OKC
def getOpponentABB(matchup):
    return matchup[-3:]

# Function that returns the score of the opponent team from our team PTS and the PLUS_MINUS. Ex: PTS=101, PLUS_MINUS=-2 => 103
def getOpponentScore(PTS,PLUS_MINUS):
    return int(PTS + PLUS_MINUS)

# Function that return if the game was played at home (H) or away (A) using matchup info. Ex: LAC vs. OKC => H, LAC @ ATL => A. 
def getHAValue(matchup):
    return 'A' if '@' in matchup else 'H'

def loadGamesDF(games):
    columns = ['TEAM_ID','TEAM_ABBREVIATION','TEAM_NAME','GAME_ID','GAME_DATE','MATCHUP','WL','PTS','PLUS_MINUS']
    gamesDF = pd.DataFrame(games)[columns]

    gamesDF = gamesDF.rename(columns={'PTS':'LAC_PTS'})
    gamesDF['OPP_TEAM_ABBREVIATION'] = gamesDF['MATCHUP'].apply(getOpponentABB)
    gamesDF['OPP_PTS'] = gamesDF.apply(lambda x: getOpponentScore(x['LAC_PTS'],x['PLUS_MINUS']*-1),axis=1)
    gamesDF['HA'] = gamesDF['MATCHUP'].apply(getHAValue)

    return gamesDF

gamesDF = loadGamesDF(games)
gamesDF

,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,LAC_PTS,PLUS_MINUS,OPP_TEAM_ABBREVIATION,OPP_PTS,HA
0,1610612746,LAC,LA Clippers,0022401036,2025-03-23,LAC vs. OKC,L,101,-2.0,OKC,103,H
1,1610612746,LAC,LA Clippers,0022401023,2025-03-21,LAC vs. MEM,W,128,20.0,MEM,108,H
2,1610612746,LAC,LA Clippers,0022400997,2025-03-18,LAC vs. CLE,W,132,13.0,CLE,119,H
3,1610612746,LAC,LA Clippers,0022400538,2025-03-16,LAC vs. CHA,W,123,35.0,CHA,88,H
4,1610612746,LAC,LA Clippers,0022400960,2025-03-14,LAC @ ATL,W,121,23.0,ATL,98,A
...,...,...,...,...,...,...,...,...,...,...,...,...
66,1610612746,LAC,LA Clippers,0022400131,2024-10-31,LAC vs. PHX,L,119,-6.0,PHX,125,H
67,1610612746,LAC,LA Clippers,0022400127,2024-10-30,LAC vs. POR,L,105,-1.0,POR,106,H
68,1610612746,LAC,LA Clippers,0022400101,2024-10-27,LAC @ GSW,W,112,8.0,GSW,104,A
69,1610612746,LAC,LA Clippers,0022400087,2024-10-26,LAC @ DEN,W,109,5.0,DEN,104,A


In [11]:
from nba_api.stats.endpoints import playbyplay

# We extract the GAME ID for each Clippers game.
gamesID = [game['GAME_ID'] for game in games]

df = playbyplay.PlayByPlay(gamesID[0]).get_data_frames()[0][['GAME_ID','EVENTMSGTYPE','PERIOD','PCTIMESTRING','HOMEDESCRIPTION','NEUTRALDESCRIPTION','VISITORDESCRIPTION','SCORE','SCOREMARGIN']]
df = df[df['SCORE'].notnull()]
df

,GAME_ID,EVENTMSGTYPE,PERIOD,PCTIMESTRING,HOMEDESCRIPTION,NEUTRALDESCRIPTION,VISITORDESCRIPTION,SCORE,SCOREMARGIN
14,0022401036,1,1,10:37,None,None,Hartenstein 1' Cutting Dunk Shot (2 PTS) (Gilg...,2 - 0,-2
21,0022401036,1,1,9:58,Zubac Putback Dunk (2 PTS),None,None,2 - 2,TIE
22,0022401036,1,1,9:50,None,None,Hartenstein 2' Cutting Layup Shot (4 PTS) (Gil...,4 - 2,-2
23,0022401036,1,1,9:30,Zubac 3' Driving Reverse Layup (4 PTS),None,None,4 - 4,TIE
32,0022401036,1,1,8:10,None,None,Wallace 26' 3PT Jump Shot (3 PTS) (Hartenstein...,7 - 4,-3
...,...,...,...,...,...,...,...,...,...
454,0022401036,18,4,1:01,None,Instant Replay4th Period (11:30 PM EST),None,101 - 100,-1
462,0022401036,3,4,0:03,None,None,Gilgeous-Alexander Free Throw 1 of 2 (25 PTS),102 - 100,-2
463,0022401036,3,4,0:03,None,None,Gilgeous-Alexander Free Throw 2 of 2 (26 PTS),103 - 100,-3
465,0022401036,3,4,0:00,Leonard Free Throw 1 of 2 (25 PTS),None,None,103 - 101,-2
